# 1. Import of input data

In [1]:
import nbimporter
import sys
import os
from pathlib import Path

In [2]:
#Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 SolarThermal.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 SolarThermal.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions may not be available.")

📁 SolarThermal.ipynb executed standalone — using directory: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Energy_systems
📘 Importing libraries from: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Libraries.ipynb
📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw


In [3]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'Solar thermal system')

# 2. Input parameters

In [4]:
Power = energysystem['Thermal capacity [kW]']
location = energysystem['Location']

solarirradiance = np.where(
    location == 'North-Italy', 0.8,
    np.where(location == 'Centre-Italy', 1.0,
    np.where(location == 'South-Italy', 1.2, np.nan))
) #kW/m2

solarefficiency = 0.6
lifespanST = 21 #years
lifespanStorage = 40 #years
TotEnergyNeeds = energysystem["Producibility [Insert the value in kWh/ year]"]
surface = (Power) / (solarefficiency * solarirradiance)
storage = (50 * surface) / 2000 #Rule of thumb 50 l /m2
scalingfactor = surface / 12.3 #ecoinvent data

# 3. Life cycle inventories

## 3.1 Module A1-3

In [5]:
SolarThermal_solarcollector = scalingfactor
SolarThermal_storage = storage #2000 l storage capacity from ecoinvent dataset

## 3.2 Module A4

In [6]:
mass = 532.8 * scalingfactor #kg primary data

SolarThermal_Truk_16_32 = ((500 + 200)*(mass)/1000)
SolarThermal_Light_Truk = (50*(mass)/1000)

## 3.3 Module B2

In [7]:
mass = np.array(mass)
lifespanST = np.array(lifespanST)
SolarThermal_Light_Truk_B2 = 50 * (lifespanST/2) * 0.01 * (mass/1000)

## 3.4 Module B4

In [8]:
SolarThermalNumberReplacements = (np.ceil(np.divide(ReqSL, lifespanStorage) - 1))
SFSolarThermal = (RSP / ReqSL)  # Scaling factor

## 3.5 Module B6

In [9]:
SolarThermal_electricity = 0.00015579 * scalingfactor

## 3.6 Module C2

In [10]:
SolarThermal_Transport_to_treatment_plant = 50 * (mass/1000)

## 3.7 Module C3

In [11]:
SolarThermal_EOL_copper = 16 * scalingfactor
SolarThermal_EOL_steel = (16 + 1.5) * scalingfactor
SolarThermal_EOL_stainlesssteel = (5 + 261) * scalingfactor
SolarThermal_EOL_glass = 9.12 * scalingfactor
SolarThermal_EOL_mineralwool = 2.43 * scalingfactor
SolarThermal_EOL_paperboard = 3.68 * scalingfactor
SolarThermal_EOL_plastic = 0.790 * scalingfactor
SolarThermal_EOL_elastomer = 5.2 * scalingfactor

## 3.8 Module C4

In [12]:
SolarThermal_EOL_landfill_copper = SolarThermal_EOL_copper
SolarThermal_EOL_landfill_steel = SolarThermal_EOL_steel
SolarThermal_EOL_landfill_stainlessSteel = SolarThermal_EOL_stainlesssteel * LandfillMetals2
SolarThermal_EOL_landfill_glass = SolarThermal_EOL_glass * LandfillGlass #kg
SolarThermal_EOL_landfill_mineralwool = SolarThermal_EOL_mineralwool * LandfillMineralWool
SolarThermal_EOL_inc_paperboard = SolarThermal_EOL_paperboard * IncinerationCardboard
SolarThermal_EOL_landfill_paperboard = SolarThermal_EOL_paperboard * LandfillCardboard
SolarThermal_EOL_inc_plastic = SolarThermal_EOL_plastic * IncinerationPEHD
SolarThermal_EOL_landfill_plastic = SolarThermal_EOL_plastic * LandfillPEHD
SolarThermal_EOL_inc_elastomer = SolarThermal_EOL_elastomer * IncinerationElastomer
SolarThermal_EOL_landfill_elastomer = SolarThermal_EOL_elastomer * LandfillElastomer

## 3.9 Module D1 (related to the export of secondary materials)

In [13]:
SolarThermal_EOL_recycling_copper = (RecyclingMetals * RecyclingEfficiencyMetals * SolarThermal_EOL_copper) - (RecycledContentMetals * SolarThermal_EOL_copper)
SolarThermal_EOL_primary_copper = ((RecycledContentMetals * SolarThermal_EOL_copper) - (RecyclingMetals * RecyclingEfficiencyMetals * SolarThermal_EOL_copper))* SRmetals

SolarThermal_EOL_recycling_steel = (RecyclingMetals * RecyclingEfficiencyMetals * SolarThermal_EOL_steel) - (RecycledContentMetals * SolarThermal_EOL_steel)
SolarThermal_EOL_primary_steel = ((RecycledContentMetals * SolarThermal_EOL_steel) - (RecyclingMetals * RecyclingEfficiencyMetals * SolarThermal_EOL_steel))* SRmetals

SolarThermal_EOL_recycling_stainlessSteel = (RecyclingMetals2 * RecyclingEfficiencyMetals * SolarThermal_EOL_stainlesssteel) - (RecycledContentMetals * SolarThermal_EOL_stainlesssteel)
SolarThermal_EOL_primary_stainlessSteel = ((RecycledContentMetals * SolarThermal_EOL_stainlesssteel) - (RecyclingMetals2 * RecyclingEfficiencyMetals * SolarThermal_EOL_stainlesssteel))* SRmetals

SolarThermal_EOL_recycling_paperboard = (RecyclingCardboard * RecyclingEfficiencyCardboard * SolarThermal_EOL_paperboard) - (RecycledContentCardboard * SolarThermal_EOL_paperboard)
SolarThermal_EOL_primary_paperboard = ((RecycledContentCardboard * SolarThermal_EOL_paperboard) - (RecyclingCardboard * RecyclingEfficiencyCardboard * SolarThermal_EOL_paperboard)) * SRinert

SolarThermal_EOL_recycling_plastic = (RecyclingPEHD * RecyclingEfficiencyPlastics * SolarThermal_EOL_plastic) - (RecycledContentPlastic * SolarThermal_EOL_plastic)
SolarThermal_EOL_primary_plastic = ((RecycledContentPlastic * SolarThermal_EOL_plastic) - (RecyclingPEHD * RecyclingEfficiencyPlastics * SolarThermal_EOL_plastic)) * SRinert


## 3.10 Module D3 (related to the export of energy as result of waste incineration)

In [14]:
SolarThermal_EOL_energyrecoveryheat_paperboard =  - SolarThermal_EOL_inc_paperboard * LHVCardboard * Xincheat
SolarThermal_EOL_energyrecoveryele_paperboard =  - SolarThermal_EOL_inc_paperboard * LHVCardboard * Xincele
SolarThermal_EOL_energyrecoveryheat_plastic =  - SolarThermal_EOL_inc_plastic * LHVHDPE * Xincheat
SolarThermal_EOL_energyrecoveryele_plastic =  - SolarThermal_EOL_inc_plastic * LHVHDPE * Xincele
SolarThermal_EOL_energyrecoveryheat_elastomer = - SolarThermal_EOL_inc_elastomer * LHVElastomer * Xincheat
SolarThermal_EOL_energyrecoveryele_elastomer = - SolarThermal_EOL_inc_elastomer * LHVElastomer * Xincele


# 4. Data for export

In [15]:
ThermalEnergyProduced_SolarThermal = TotEnergyNeeds